
CBOW


In [ ]:
import pandas as pd 

artigo_treino = pd.read_csv("./treino.csv")
artigo_teste = pd.read_csv("./teste.csv")

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

texto = [
    "Este produto é muito bom",
    "Este produto é muito ruim"
]

vetorizador = CountVectorizer()
vetorizador.fit(texto)
print(vetorizador.vocabulary_)

{'este': 1, 'produto': 3, 'muito': 2, 'bom': 0, 'ruim': 4}


In [14]:
# from huggingface_hub import hf_hub_download
# from safetensors.numpy import load_file

# path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
#                        filename="embeddings.safetensors")

# data = load_file(path)
# vectors = data["embeddings"]

# vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
#                              filename="vocab.txt")
# with open(vocab_path) as f:
#     vocab = [w.strip() for w in f]

# print(vectors.shape)


In [ ]:
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file
from gensim.models import KeyedVectors

# embeddings
path = hf_hub_download(
    repo_id="nilc-nlp/word2vec-cbow-300d",
    filename="embeddings.safetensors"
)
data = load_file(path)
vectors = data["embeddings"]

# vocabulário
vocab_path = hf_hub_download(
    repo_id="nilc-nlp/word2vec-cbow-300d",
    filename="vocab.txt"
)
with open(vocab_path, encoding="utf-8") as f:
    vocab = [w.strip() for w in f]

# checagem de sanidade
print(len(vocab), vectors.shape)

# Word2Vec no gensim
modelo = KeyedVectors(vector_size=vectors.shape[1])
modelo.add_vectors(vocab, vectors)


929606 (929606, 300)


In [16]:
data.keys()


dict_keys(['embeddings'])

In [17]:
modelo.most_similar("amor")


[('afeto', 0.7056611180305481),
 ('tormento', 0.6202821135520935),
 ('desamor', 0.6199557781219482),
 ('ciúme', 0.6167261600494385),
 ('arrependimento', 0.6010848879814148),
 ('encanto', 0.5992918014526367),
 ('instincto', 0.5957212448120117),
 ('ódio', 0.580238401889801),
 ('infortúnio', 0.5744850039482117),
 ('egoísmo', 0.5689378380775452)]

In [18]:
modelo.get_vector("china")

array([-1.49033e-01,  1.26020e-01,  2.17628e-01,  1.82684e-01,
        1.65151e-01, -1.59660e-01, -2.34411e-01,  6.00570e-02,
        8.03680e-02,  2.87578e-01, -4.81100e-03, -5.68800e-02,
        2.15676e-01,  8.65540e-02,  1.25983e-01,  3.36157e-01,
       -1.83254e-01, -1.18499e-01,  1.13010e-02,  1.03814e-01,
        9.37640e-02,  2.90178e-01, -1.64395e-01, -1.13300e-02,
       -1.80676e-01, -1.15820e-02,  1.08728e-01,  1.65898e-01,
        9.37900e-02,  2.66767e-01, -1.29890e-02,  9.16030e-02,
        2.21292e-01, -1.36497e-01, -4.26350e-02, -1.30038e-01,
        2.17067e-01, -1.01963e-01, -3.70960e-02,  1.42155e-01,
        3.41109e-01,  2.46560e-01,  1.27458e-01,  5.72360e-02,
       -1.47962e-01, -1.60290e-02,  1.86533e-01,  7.71550e-02,
       -3.50024e-01, -4.06085e-01,  1.67131e-01, -4.75230e-02,
        5.13780e-02, -1.28224e-01,  1.06580e-02, -2.92652e-01,
        1.40540e-01, -4.57049e-01,  1.31094e-01,  2.03234e-01,
        2.94019e-01,  7.38370e-02,  1.11554e-01, -1.642

In [19]:
modelo.most_similar("china")

[('rússia', 0.7320705056190491),
 ('índia', 0.7241616249084473),
 ('tailândia', 0.701935887336731),
 ('indonésia', 0.6860769987106323),
 ('turquia', 0.6741335988044739),
 ('malásia', 0.6665689945220947),
 ('mongólia', 0.6593616008758545),
 ('manchúria', 0.658184826374054),
 ('urss', 0.6581669449806213),
 ('grã-bretanha', 0.6568097472190857)]

In [20]:
modelo.most_similar(positive= ["china","brasil"])

[('japão', 0.725788950920105),
 ('ocidente', 0.5933547019958496),
 ('canadá', 0.5921916961669922),
 ('austrália', 0.5688517093658447),
 ('tibete', 0.5675076246261597),
 ('haiti', 0.5656365752220154),
 ('irã', 0.5653263330459595),
 ('camboja', 0.5621733069419861),
 ('índia', 0.5570496916770935),
 ('rússia', 0.5542328953742981)]

In [21]:
modelo.most_similar(positive= ["carros","foguete"], negative="carro")

[('foguetes', 0.5930031538009644),
 ('foguetões', 0.5570027828216553),
 ('caças', 0.550533652305603),
 ('aviões', 0.5263278484344482),
 ('satélites', 0.5231359601020813),
 ('motores', 0.51063472032547),
 ('hidroaviões', 0.4993920922279358),
 ('vaivéns', 0.48650243878364563),
 ('veículos', 0.4795992970466614),
 ('barcos', 0.47358769178390503)]

# Classificador

In [22]:
import nltk
import string

def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerica =[]
    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.punctuation: continue
        lista_alfanumerica.append(token_valido)

    return lista_alfanumerica    

In [23]:
tokenizador("texto com, pontuação")

['texto', 'com', 'pontuação']

In [1]:
import numpy as np

def soma_errada():
    v = np.zeros(3)
    for x in [np.array([1, 2, 3]), np.array([4, 5, 6])]:
        v =+ x
    return v

def soma_correta():
    v = np.zeros(3)
    for x in [np.array([1, 2, 3]), np.array([4, 5, 6])]:
        v += x
    return v

print("ERRADO (=+):", soma_errada())
print("CERTO (+=):", soma_correta())


ERRADO (=+): [4 5 6]
CERTO (+=): [5. 7. 9.]


In [ ]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras_numeros):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try: # trataemnto de palavras ou números que não contem no conjunto de dados 
      vetor_resultante += modelo.get_vector(pn) #CORREEÇÃO FEITA DE ACORDO COM A FUNÇÃO ACIMA += E NÃO =+
    except KeyError:
      if pn.isnumeric(): #numeros 
        pn = "0"*len(pn) # pega o valor e coloca 0 ent 123 fica 000 preservando o tamanho
        vetor_resultante += modelo.get_vector(pn)
      else: # palavras
        vetor_resultante += modelo.get_vector('unknown')
  return vetor_resultante

In [25]:
palavras_numeros = tokenizador("Texto fiapsa 12")
vetor_texto = combinacao_de_vetores_por_soma(palavras_numeros)
print(len(vetor_texto))
print(vetor_texto)

300
[-0.014933 -0.031461 -0.020224 -0.020854 -0.022564  0.041807 -0.048768
 -0.007095  0.054534 -0.002426  0.005624 -0.020128  0.023129 -0.026669
  0.01469   0.00961   0.022168  0.002709  0.043822 -0.015186  0.052928
  0.014309  0.015711 -0.016347  0.005049 -0.020506 -0.056819 -0.006213
 -0.042197  0.003012 -0.036475  0.037598 -0.012933 -0.018753 -0.056046
  0.001505 -0.012291 -0.02913   0.003446 -0.032035  0.006458 -0.016397
 -0.03731  -0.007289 -0.060034  0.029378 -0.03731   0.016922 -0.032621
 -0.03493  -0.027019 -0.027014 -0.016002  0.030993  0.021334  0.064816
 -0.009871  0.02216  -0.026325 -0.000802  0.013731 -0.008782  0.039407
  0.042168  0.029307 -0.017424  0.012714 -0.054174  0.030855 -0.034067
 -0.032796  0.029321 -0.049295 -0.004273 -0.050534  0.040576 -0.002042
 -0.002602  0.026559 -0.010416 -0.053731 -0.039648  0.05321   0.01027
  0.014883 -0.034597 -0.001676 -0.015215 -0.065981 -0.041821 -0.03963
 -0.034798 -0.006172  0.027046  0.022387 -0.014069  0.022973  0.004068
 -0.

In [29]:
def matriz_vetores(textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i]= combinacao_de_vetores_por_soma(palavras_numeros)
    return matriz

In [31]:
matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)

In [27]:
artigo_teste.head()

,title,text,date,category,subcategory,link
0,Grandes irmãos,"RIO DE JANEIRO - O Brasil, cada vez menos famí...",2017-03-06,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...
1,Haddad congela orçamento e suspende emendas de...,"O prefeito de São Paulo, Fernando Haddad (PT),...",2016-08-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
2,Proposta de reforma da Fifa tem a divulgação d...,"A Fifa divulgou, nesta quinta (10), um relatór...",2015-10-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...
3,"Mercado incipiente, internet das coisas conect...","Bueiros, coleiras, aparelhos hospitalares, ele...",2016-11-09,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/09/1...
4,"Mortes: Psicanalista, estudou o autismo em cri...",Toda vez que o grupo de amigos de Silvana Rabe...,2017-02-07,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/07...


In [33]:
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

(90000, 300)
(20513, 300)


In [34]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(matriz_vetores_treino,artigo_treino.category)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [35]:
LR.score(matriz_vetores_teste,artigo_teste.category)

0.5134305074830595

In [36]:
artigo_teste.category.unique()

array(['colunas', 'esporte', 'mercado', 'cotidiano', 'mundo', 'ilustrada'],
      dtype=object)

In [44]:
from sklearn.metrics import classification_report

label_prevista = LR.predict(matriz_vetores_teste)
CR = classification_report(artigo_teste.category,label_prevista)
print(CR)

              precision    recall  f1-score   support

     colunas       0.62      0.39      0.47      6103
   cotidiano       0.33      0.54      0.41      1698
     esporte       0.67      0.56      0.61      4663
   ilustrada       0.03      0.42      0.06       131
     mercado       0.63      0.58      0.60      5867
       mundo       0.40      0.59      0.48      2051

    accuracy                           0.51     20513
   macro avg       0.45      0.51      0.44     20513
weighted avg       0.59      0.51      0.54     20513



In [45]:
from sklearn.dummy import DummyClassifier
dc = DummyClassifier()
dc.fit(matriz_vetores_teste,artigo_teste.category)
label_prevista_dc = dc.predict(matriz_vetores_teste)
CR_dc = classification_report(artigo_teste.category,label_prevista)
print(CR_dc)


              precision    recall  f1-score   support

     colunas       0.62      0.39      0.47      6103
   cotidiano       0.33      0.54      0.41      1698
     esporte       0.67      0.56      0.61      4663
   ilustrada       0.03      0.42      0.06       131
     mercado       0.63      0.58      0.60      5867
       mundo       0.40      0.59      0.48      2051

    accuracy                           0.51     20513
   macro avg       0.45      0.51      0.44     20513
weighted avg       0.59      0.51      0.54     20513



# SKIP GRAM

In [46]:
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file

path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                       filename="embeddings.safetensors")

data = load_file(path)
vectors = data["embeddings"]

vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                             filename="vocab.txt")
with open(vocab_path) as f:
    vocab = [w.strip() for w in f]

print(vectors.shape)


embeddings.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

(929606, 300)


In [ ]:
modelo_skipgram = KeyedVectors(vector_size=vectors.shape[1]) #300
modelo_skipgram.add_vectors(vocab, vectors)

In [ ]:
import numpy as np

def combinacao_de_vetores_por_soma_skipgram(palavras_numeros):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try: # trataemnto de palavras ou números que não contem no conjunto de dados 
      vetor_resultante += modelo_skipgram.get_vector(pn)
    except KeyError:
      if pn.isnumeric(): #numeros 
        pn = "0"*len(pn) # pega o valor e coloca 0 ent 123 fica 000 preservando o tamanho
        vetor_resultante += modelo_skipgram.get_vector(pn)
      else: # palavras
        vetor_resultante += modelo_skipgram.get_vector('unknown')
  return vetor_resultante

In [54]:
def matriz_vetores_skipgram(textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i]= combinacao_de_vetores_por_soma_skipgram(palavras_numeros)
    return matriz

In [55]:
matriz_vetores_treino_skipgram = matriz_vetores_skipgram(artigo_treino.title)
matriz_vetores_teste_skipgram = matriz_vetores_skipgram(artigo_teste.title)

In [56]:
artigo_teste.head()

,title,text,date,category,subcategory,link
0,Grandes irmãos,"RIO DE JANEIRO - O Brasil, cada vez menos famí...",2017-03-06,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...
1,Haddad congela orçamento e suspende emendas de...,"O prefeito de São Paulo, Fernando Haddad (PT),...",2016-08-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
2,Proposta de reforma da Fifa tem a divulgação d...,"A Fifa divulgou, nesta quinta (10), um relatór...",2015-10-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...
3,"Mercado incipiente, internet das coisas conect...","Bueiros, coleiras, aparelhos hospitalares, ele...",2016-11-09,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/09/1...
4,"Mortes: Psicanalista, estudou o autismo em cri...",Toda vez que o grupo de amigos de Silvana Rabe...,2017-02-07,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/07...


In [57]:
print(matriz_vetores_treino_skipgram.shape)
print(matriz_vetores_teste_skipgram.shape)

(90000, 300)
(20513, 300)


In [58]:
from sklearn.linear_model import LogisticRegression

LR_skipgram = LogisticRegression()
LR_skipgram.fit(matriz_vetores_treino_skipgram,artigo_treino.category)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [62]:
LR_skipgram.score(matriz_vetores_teste_skipgram,artigo_teste.category)

0.5274703846341344

In [60]:
artigo_teste.category.unique()

array(['colunas', 'esporte', 'mercado', 'cotidiano', 'mundo', 'ilustrada'],
      dtype=object)

In [ ]:
from sklearn.metrics import classification_report

label_prevista_skipgram = LR_skipgram.predict(matriz_vetores_teste_skipgram)
CR_skipgram = classification_report(artigo_teste.category,label_prevista_skipgram)
print(CR_skipgram)

              precision    recall  f1-score   support

     colunas       0.62      0.40      0.49      6103
   cotidiano       0.35      0.55      0.43      1698
     esporte       0.68      0.59      0.63      4663
   ilustrada       0.03      0.45      0.06       131
     mercado       0.64      0.58      0.61      5867
       mundo       0.43      0.59      0.50      2051

    accuracy                           0.53     20513
   macro avg       0.46      0.53      0.45     20513
weighted avg       0.59      0.53      0.55     20513

